In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [62]:
class CNN_nn(nn.Module):
    def __init__(self):
        super(CNN_nn, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(16, 32, 5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        
        self.linear = nn.Linear(4*4*32, 10)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.pool2(self.relu2(self.conv2(out)))
        out = out.view(out.size(0), -1)        
        self.linear(out)
        return out

In [63]:
class CNN_F(nn.Module):
    def __init__(self):
        super(CNN_F, self).__init__()
        
        self.cnn1_weight = nn.Parameter(torch.rand(16,1,5,5))
        self.bias1_weight = nn.Parameter(torch.rand(16))
        
        self.cnn2_weight = nn.Parameter(torch.rand(32,16,5,5))
        self.bias2__weight = nn.Parameter(torch.rand(32))
        
        self.linear_weight = nn.Parameter(torch.rand(4*4*32,10))
        self.bias3_weight = nn.Parameter(torch.rand(10))
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = F.conv2d(x, self.cnn1_weight, self.bias1_weight)
        out = F.relu(out)
        out = F.max_pool2d(out)
        
        out = F.max_pool2d(F.relue(F.conv2d(x, 
                        self.cnn2_weight, self.bias2__weight)))
        
        out = F.linear(out, self.linear_weight, self.bias3_weight)
        return out

In [64]:
class CNN_Sequential(nn.Module):
    def __init__(self):
        super(CNN_Sequential, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)   # (batch, 32, 7, 7)
        x = x.view(x.size(0), -1)   # (batch, 32*7*7)
        output  = self.out(x)
        return output, x

In [65]:
cnn_nn = CNN_nn()
cnn_f = CNN_F()
cnn_seq = CNN_Sequential()

print("CNN_nn:", cnn_nn)
print("-----"*6)
print("CNN_functional:", cnn_f)
# print(list(cnn_f.parameters()))
print("-----"*6)
print("CNN_Sequential:", cnn_seq)
print("-----"*6)

CNN_nn: CNN_nn(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=512, out_features=10, bias=True)
)
------------------------------
CNN_functional: CNN_F()
------------------------------
CNN_Sequential: CNN_Sequential(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)
------------------------------


In [66]:
optimizer_NN = torch.optim.Adam(cnn_nn.parameters())
optimizer_F = torch.optim.Adam(cnn_f.parameters())
optimizer_Seq = torch.optim.Adam(cnn_seq.parameters())

loss_func_NN = nn.CrossEntropyLoss()
loss_func_F = nn.CrossEntropyLoss()
loss_func_Seq = nn.CrossEntropyLoss()

In [67]:
inputs = torch.randn(1,1, 240, 240)
print(inputs.size())
target = torch.randn(10)
print(target.size())

for _ in range(10):
    optimizer_NN.zero_grad()
    optimizer_F.zero_grad()
    optimizer_Seq.zero_grad()
    
    output_nn = cnn_nn(inputs)
    output_f = cnn_f(inputs)
    output_seq = cnn_seq(inputs)
    
    loss_nn = loss_func_NN(output_nn, target)
    loss_f = loss_func_F(output_f, target)
    loss_seq = loss_func_Seq(output_seq, target)
    
    loss_nn.backward()
    loss_f.backward()
    loss_seq.backward()
    
    optimizer_NN.step()
    optimizer_F.step()
    optimizer_Seq.step()

torch.Size([1, 1, 240, 240])
torch.Size([10])


RuntimeError: size mismatch, m1: [1 x 103968], m2: [512 x 10] at /tmp/pip-req-build-ufslq_a9/aten/src/TH/generic/THTensorMath.cpp:136

In [55]:
test = torch.randn(1,3,2)
print(test.size())
print(test.size(0))
print(test.size(1))

torch.Size([1, 3, 2])
1
3
